### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
csvpath = os.path.join("Resources","purchase_data.csv")
print(csvpath)


# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csvpath,delimiter=",")

Resources/purchase_data.csv


In [13]:
#Define function to use in the program to change the format to currency and percentage
def currency_format(x):
     return "${:,.2f}".format(x)    

def percentage_format(x):
     return round((x * 100),2).astype(str) + '%'



## Player Count

* Display the total number of players


In [14]:
#count the number of unique players
players = purchase_data["SN"].value_counts()
total_players = pd.DataFrame({len(players)})
total_players.columns = ['Total Players']
total_players.head()

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [15]:
#Count the number of unique items and save it to unique_items variable
unique_items = len(purchase_data["Item ID"].unique())
unique_items

#Calculate the average price and use the currency format function to save avg_price variable
avg_price = currency_format (purchase_data["Price"].mean())

#Count the number of purchases
number_purchases = len(purchase_data["Purchase ID"].unique())

#Sum the total revenue
total_revenue = currency_format ( purchase_data["Price"].sum())

summary_df = pd.DataFrame({
    'Number of Unique Items': [unique_items],
    'Average Price': [avg_price],
    'Number of Purchases': [number_purchases],
    'Total Revenue': [total_revenue]
})
summary_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [16]:
#Get the unique list of players with gender removing duplicates
unique_players = purchase_data.drop_duplicates(subset =["SN","Gender"])

#Count the number of players by gender inside a data frame
gender = pd.DataFrame(unique_players["Gender"].value_counts())

#Calculate percentage and set format with function
percentage_gender = percentage_format((gender["Gender"]/ gender["Gender"].sum()).astype(float))

#Add variable to dataframe
gender["Percentage of Players"] = percentage_gender
gender


,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [17]:
#group data by Gender variable
grouped_purchases = purchase_data.groupby(["Gender"])

#Count the purchases by gender
purchase_count = pd.DataFrame(grouped_purchases["Purchase ID"].count())

#calculate the sum of all the and divide the by the number of purchases
average_purchase_price = pd.DataFrame((grouped_purchases["Price"].sum() / grouped_purchases["Price"].count()).map('${:,.2f}'.format))

#calculate all the purchase value
total_purchase_vale = pd.DataFrame(grouped_purchases["Price"].sum().map('${:,.2f}'.format))

#sum the number of prices by gender and divide them by the number of unique persons defined in previous cell
average_purchase_per_person = pd.DataFrame((grouped_purchases["Price"].sum() / gender["Gender"]).map('${:,.2f}'.format))

#add all data to the first dataframe
purchase_count ["Average Purchase Price"] = average_purchase_price
purchase_count ["Total Purchase Value"] = total_purchase_vale
purchase_count ["Avg Total Purchase per Person"] = average_purchase_per_person

purchase_count.head()


,Purchase ID,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [18]:
#eliminate duplicayes to keep Player and Age
unique_players_age = purchase_data.drop_duplicates(subset =["SN","Age"])

#Set the lenght of each bin
cut_bins = [0,10,15,20,25,30,35,40,200]

#Use cut function to count player age per bin using the unique players-age df
bins = pd.DataFrame(pd.cut(unique_players_age['Age'], bins=cut_bins, right = False, ordered = False, labels=["<10", "10-14", "15-19","20-24","25-29","30-34","35-39","40+"]).value_counts()).rename(columns = {"Age": "Total Count"})

#Count the percentage of players per bin and add them to the Result DataFrame
percentage_players = pd.DataFrame(percentage_format(bins["Total Count"] / bins["Total Count"].sum()))
bins["Percentage of Players"] = percentage_players
bins


,Total Count,Percentage of Players
20-24,258,44.79%
15-19,107,18.58%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
10-14,22,3.82%
<10,17,2.95%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [19]:
#set bins of age with the cut function from all data
purchasing_analysis = pd.DataFrame(pd.cut(purchase_data["Age"],bins=cut_bins, right = False, ordered = False, labels=["<10", "10-14", "15-19","20-24","25-29","30-34","35-39","40+"]))

#Add the age bins to the original DF for analysis
purchase_data ["Age_Bin"] = purchasing_analysis

#Group by all date per Age Bin for simpler analysis
grouped_purchases = purchase_data.groupby(["Age_Bin"])                            

#Count purchases per bin
purchase_count = pd.DataFrame(grouped_purchases["Age_Bin"].count()).rename(columns = {"Age_Bin": "Purchase Count"})

#sum all prices per bin
total_purchase_value = pd.DataFrame(grouped_purchases["Price"].sum().map('${:,.2f}'.format)).rename(columns = {"Price": "Total Purchase Value"})


#Calculate the average purchase price
average_purchase_price = pd.DataFrame((grouped_purchases["Price"].sum() / grouped_purchases["Age_Bin"].count()).map('${:,.2f}'.format))

#Calculate the avg purchase per person using df from previous cell
avg_purchase_per_person = pd.DataFrame((grouped_purchases["Price"].sum() / bins["Total Count"]).map('${:,.2f}'.format))

#Add all variables to DataFrame
purchase_count ["Average Purchase Price"] = average_purchase_price
purchase_count ["Total Purchase Value"] = total_purchase_value
purchase_count ["Avg Total Purchase per Person"] = avg_purchase_per_person

purchase_count


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age_Bin,,,,
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [20]:
#create database grouped by player
grouped_by_player = purchase_data.groupby(["SN"]) 

#count all purchases made by each player using grouped DF
purchase_by_player = pd.DataFrame(grouped_by_player["SN"].count()).rename(columns = {"SN": "Purchase Count"})

#sum all prices from purchases made by each player
total_purchase_by_player = pd.DataFrame(grouped_by_player["Price"].sum()).rename(columns = {"Price": "Total Purchase Value"})

#get the average pruchase price per player
average_purchase_by_player = pd.DataFrame((grouped_by_player["Price"].sum() / grouped_by_player["SN"].count()))

#adding all variables to a unique DF
purchase_by_player["Average Purchase Price"] = average_purchase_by_player
purchase_by_player["Total Purchase Value"] = total_purchase_by_player

#Sorting by total purchase value per player y descending order
purchase_by_player.sort_values(by=['Total Purchase Value'], ascending = False, inplace = True)

format_dict = {"Average Purchase Price": '${:,.2f}', "Total Purchase Value": '${:,.2f}'}

purchase_by_player.head().style.format(format_dict)




,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [21]:
#Get only the 3 needed columns
popular_items = purchase_data[["Item ID", "Item Name", "Price"]]

#Create a Grouped DF by two indexes
grouped_popular_items = popular_items.groupby(["Item ID","Item Name"])

#Count the purchases made by each item
popular_items_count = pd.DataFrame(grouped_popular_items["Item ID"].count()).rename(columns = {"Item ID": "Purchase Count"})

#Get the average price by each item
popular_items_avg_price = pd.DataFrame(grouped_popular_items["Price"].sum() / grouped_popular_items["Item ID"].count())

#Get the total purchased value by each item
popular_items_purchase_value = pd.DataFrame(grouped_popular_items["Price"].sum())

#add data to a single dataframe
popular_items_count["Item Price"] = popular_items_avg_price
popular_items_count["Total Purchase Value"] = popular_items_purchase_value

#Sort the dataframe by the total value in descending order
popular_items_count.sort_values(by=['Purchase Count'], ascending = False, inplace = True)

#define the formatting dictionary for the final print
format_dict = {"Item Price": '${:,.2f}', "Total Purchase Value": '${:,.2f}'}

#apply style and print only top 5 records
popular_items_count.head().style.format(format_dict)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [22]:
#Sort the dataframe by item price in descending order
most_profitable_items = popular_items_count.sort_values(by=['Total Purchase Value'], ascending = False)

#define the formatting dictionary for the final print
format_dict = {"Item Price": '${:,.2f}', "Total Purchase Value": '${:,.2f}'}

#apply style and print only top 5 records
most_profitable_items.head().style.format(format_dict)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
